# Tercera parte.  Recomendacion basada en filtrado colaborativo.

**Grupo 16**
- Daniela Alejandra Cordova Porta
- David Bugoi
- Erik Karlgren Domercq

En esta tercera parte utilizaremos la librería SURPRISE Se puede consultar la documentacion en http://surpriselib.com/

Para instalarla: `conda install -c conda-forge scikit-surprise` o `pip install numpy pip install scikit-surprise`.

La librería SurPRISE (<i>Simple Python RecommendatIon System Engine</i>) tiene algoritmos de predición de ratings incluidos: <i>baseline algorithms</i>, <i>neighborhood methods</i>, <i>matrix factorization-based</i> (SVD, PMF, SVD++, NMF) y otros. También tiene predefinidas las medidas de similitud más comunes sobre vectores (<i>cosine</i>, MSD, pearson…) Una de las cosas más útiles es que proporciona herramientas para evaluar, analizar y comparar el rendimiento de distintos algoritmos. Lo que vamos a hacer en esta parte de la práctica es probar varios procedimientos de evaluación cruzada midiendo datos sobre errores entre el valor real (conocido) y la predicción del recomendador. Las siglas corresponden a las siguientes medidas:

- MAE: _Mean Absolute Error_
- RMSE: _Root mean square error_ (RMSE)
- MSE: mean square error is defined as the expected value of the square of the difference between the estimator and the parameter. -square root of the mean square error.

Vamos a ejecutar algunos recomendadores y evaluarlos para poder comentar los resultados.

In [7]:
from collections import defaultdict
import numpy as np

from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [8]:
## Ejemplo getting started de la documentación de SURPRISE
##http://surpriselib.com/

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9316  0.9358  0.9376  0.9398  0.9338  0.9357  0.0028  
MAE (testset)     0.7347  0.7388  0.7369  0.7410  0.7368  0.7377  0.0021  
Fit time          3.65    3.64    3.66    3.62    3.74    3.66    0.04    
Test time         0.14    0.10    0.13    0.11    0.18    0.13    0.03    


{'test_rmse': array([0.93163571, 0.93575979, 0.93756398, 0.93980909, 0.9338424 ]),
 'test_mae': array([0.73472964, 0.73875508, 0.73693932, 0.74104354, 0.73683877]),
 'fit_time': (3.6474547386169434,
  3.63614821434021,
  3.6558492183685303,
  3.6166446208953857,
  3.7376270294189453),
 'test_time': (0.1361072063446045,
  0.10495305061340332,
  0.1336040496826172,
  0.11096739768981934,
  0.1831073760986328)}

## Definición de funciones
Usaremos estas funciones para evaluar varios algoritmos de recomendación. La función `get_results` escribirá sobre el fichero `results_user_cf.csv` añadiéndole texto al final. Queremos asegurarnos de que el fichero exista y esté vacío (salvo por una cabecera) antes de llamar a dicha función por primera vez.

In [9]:
# Creamos el fichero en el mismo directorio que está guardado este notebook y le añadimos una cabecera
f = open("results_user_cf.csv", 'w')
f.write("K,Algorithm,Precision,Recall,F1\n")
f.close()

In [10]:
# Evaluation extracted from surprise: 
# https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k
def measures_at_k(predictions, k, th_recom, th_relev):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    onehits = dict()
    mrr = dict()
    
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= th_relev) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= th_recom) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= th_relev) and (est >= th_recom))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
       
        
    return precisions, recalls


In [11]:
def f1(precision, recall):
    """
        Funcion que calcula el f1 (media armónica) en funcion de precision y recall
    """
    denominador = precision + recall
    
    if denominador == 0:
        return 0
    else:
        return (2 * precision * recall) / denominador

In [12]:
# Guarda los resultados de las recomendaciones en el fichero "results_user_cf.csv"
def get_results(recommendations, k, knn):
    """
        Function to get the measures results 
    """
    # threshold = 4 --> solo se tienen en cuenta peliculas que hayan 
    # sido puntuadas con 4 o 5 estrellas
    precisions, recalls  = measures_at_k(recommendations, k, th_recom=4, th_relev=1)
    
    # Measures can then be averaged over all users
    precision_result = sum(prec for prec in precisions.values()) / len(precisions)
    recall_result = sum(rec for rec in recalls.values()) / len(recalls)
    # Media armónica  
    f1_result = f1(precision_result, recall_result)
    # En este archivo se volcarán los resultados de las métricas. Tiene que existir. 
    f = open("results_user_cf.csv", 'a')
    #f = open("C:/hlocal/results_user_cf.csv", 'a')
    f.write(str(k) + ',' + knn + "," + str(precision_result) + ',' + str(recall_result) + ',' +  str(f1_result) +  '\n') 
    f.close()
    

## Evaluación de los algoritmos de recomendación
Hemos cargado antes los datos de movieLens para 100K con la siguiente función:
```python
data = Dataset.load_builtin('ml-100k')
```
Ahora creamos 2 conjuntos de datos: los datos de entrenamiento (`training_set`) y los de evaluación (`evaluation_set`). Cada uno contendrá la mitad de los datos.

In [13]:
training_set, evaluation_set = train_test_split(data, test_size=.5)

Ahora vamos a emplear varios algoritmos de recomendación y guardar sus resultados en el fichero `results_user_cf.csv`.
### KNN Basic

In [15]:
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [17]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [18]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [11]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

In [16]:
cross_validate(recommendation_algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0043  1.0006  0.9993  1.0014  0.9999  1.0011  0.0017  
MAE (testset)     0.7972  0.7906  0.7910  0.7889  0.7940  0.7924  0.0029  
Fit time          1.66    1.67    1.70    1.67    1.68    1.68    0.01    
Test time         3.73    3.42    3.45    3.76    3.56  

{'test_rmse': array([1.00429252, 1.00055043, 0.99929365, 1.00144132, 0.99990104]),
 'test_mae': array([0.79724739, 0.7906253 , 0.79104514, 0.78889743, 0.79403654]),
 'fit_time': (1.6617188453674316,
  1.6731925010681152,
  1.700601577758789,
  1.6683690547943115,
  1.6756517887115479),
 'test_time': (3.732666254043579,
  3.42401123046875,
  3.447269916534424,
  3.756788730621338,
  3.555835485458374)}

### KNN With Means

In [20]:
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [21]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [22]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [23]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

In [24]:
cross_validate(recommendation_algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9469  0.9418  0.9389  0.9331  0.9319  0.9385  0.0055  
MAE (testset)     0.7380  0.7338  0.7342  0.7291  0.7255  0.7321  0.0043  
Fit time          1.69    1.70    1.74    1.72    1.76    1.72    0.03    
Test time         3.67    3.69    3.85    3.85    3.

{'test_rmse': array([0.94687422, 0.94176123, 0.93885707, 0.93314358, 0.9318679 ]),
 'test_mae': array([0.73796376, 0.73380236, 0.73422473, 0.72914897, 0.72548756]),
 'fit_time': (1.6928811073303223,
  1.6972360610961914,
  1.74053955078125,
  1.7221970558166504,
  1.758854627609253),
 'test_time': (3.674858808517456,
  3.691965341567993,
  3.8530361652374268,
  3.8516950607299805,
  3.8879013061523438)}

### KNN With Z-Score

In [25]:
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [26]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [27]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [28]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

In [29]:
cross_validate(recommendation_algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9477  0.9345  0.9376  0.9409  0.9311  0.9383  0.0057  
MAE (testset)     0.7346  0.7271  0.7274  0.7328  0.7272  0.7298  0.0032  
Fit time          1.79    1.76    1.83    1.85    1.75    1.79    0.04    
Test time         4.01    4.34    4.23    3.98    4

{'test_rmse': array([0.94770425, 0.93445182, 0.9375827 , 0.94085838, 0.9310527 ]),
 'test_mae': array([0.73461205, 0.72713066, 0.72741664, 0.73277199, 0.72717478]),
 'fit_time': (1.7854182720184326,
  1.763239860534668,
  1.8270072937011719,
  1.845022439956665,
  1.7492692470550537),
 'test_time': (4.013169765472412,
  4.343783617019653,
  4.233802556991577,
  3.9780144691467285,
  3.997918128967285)}

### KNN Baseline

In [30]:
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [31]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [32]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [33]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

In [34]:
cross_validate(recommendation_algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9210  0.9178  0.9267  0.9090  0.9311  0.9211  0.0076  
MAE (testset)     0.7205  0.7186  0.7278  0.7138  0.7288  0.7219  0.0057  
Fit time          1.67    1.65    1.67    1.68    1.66    1.67    0.01    
Test time         4.52    4.56    4.52    4.63    4.3

{'test_rmse': array([0.92098501, 0.91784152, 0.92669751, 0.90904049, 0.9311187 ]),
 'test_mae': array([0.7204744 , 0.7185902 , 0.7277729 , 0.71383533, 0.72880122]),
 'fit_time': (1.674551010131836,
  1.6541361808776855,
  1.6705143451690674,
  1.6819627285003662,
  1.6604681015014648),
 'test_time': (4.517550945281982,
  4.559155225753784,
  4.5216851234436035,
  4.627239465713501,
  4.314043283462524)}

## Ejercicio:  se pide ejecutar, comprender y escribir comentarios razonados sobre la evaluación de distintos recomendadores.
    
Prueba otros algoritmos de predicción de ratings basados en la estimación de los vecinos más próximos y realiza evaluaciones de su comportamiento. Comenta los resultados.
Puedes consultar la documentación en https://surprise.readthedocs.io/en/stable/knn_inspired.html#

### MAE y RMSE

Tanto `MAE` como `RMSE` expresan el error de predicción promedio y los valores mientras más bajos son, mejores. La diferencia entre estas dos es que como RMSE consiste en elevar los errores al cuadrado antes de promediarlos, le otorga un peso relativamente alto a los errores grandes que se obtienen. 


Si analizamos los valores que dieron para cada algoritmo, desde SVD hasta KNN Baseline; podemos observar que en el orden: KNN Basic, SVD, KNN With Means, KNN With Z-Score, KNN Baseline va mejorando la media de error. Esto quiere decir que la predicción del recomendador difiere cada vez menos con los valores reales. 

Otro punto a observar es que `KNN Basic` en comparación con los otros algoritmos en el error `RMSE` crece con ritmo distinto a los demás, subiendo por encima del 1. Esto quiere decir que tiene errores más grandes en comparación con los demás algoritmos.

En resumen; si queremos un recomendador que nos devuelva unos k vecinos que se adapten mejor a las valoraciones reales que tendrá el usuario, la mejor opción según este análisis es el `KNN Baseline`.

### Recall y Precisión

Queremos recomendar al usuario los N elementos primeros en lugar de todos a un usuario. Por ello, el recall y precisión lo envaluamos sobre un k donde k es un número entero definido por el usuario para que coincida el top N de recomendaciones principales. Entonces la precisón en k es la proporción de elementos recomendados en el conjunto top k relevantes. 

Si k= 10 en un top-10 de recomendados y la precisión es un 80%, esto siginfica que el 80% de las recomendaciones que hago son relevantes para el usuario.

El Recall en k es la proporción de elementos relevantes que se encuentran en el top_k de recomendaciones.

Si k=10 en un top-10 y el recall es 40%, quiere decir que el 40% del número total de elementos relevantes aparecen en los primeros resultados.



F1 nos ayuda a entender el balance entre el recall y precisión, su máximo valor es 1 y quiere decir que todas las predicciones son recomendaciones correctas. 

Si vemos el fichero `results_user_cf.csv`, observamos que la precisión de cada algoritmo no varía en función del valor de K, pero el recall sí que aumenta al incrementarse K. Consecuentemente, la media armónica también aumenta y conseguimos mejores recomendaciones.

A la hora de comparar los algoritmos entre sí el que consigue mejor precisión es KNNBasic seguido de, en orden decreciente de precisión, KNN Baseline, KNN With Z-Score y KNN With Means. Es decir, KNN Basic es el algoritmo que mayor proporción de items relevantes consigue de los items que recomienda. No obstante, KNN Basic consigue el peor recall de todos los algoritmos que hemos probado, por lo que es el que menos items recomienda de los items relevantes. El resto tienen recalls parecidos para el mismo valor de K.

Para comparar todos los algoritmos es preferible usar la media armónica de la precisión y el recall pues cuando se incrementa la precisión decrece el recall y viceversa, así que buscamos un "equilibrio" óptimo. En este aspecto podemos observar que KNN Baseline es el algoritmo que mejor se comporta de todos, seguido de KNN With Means y KNN With Z-Score, y siendo KNN Basic el peor de todos.

#### Analicemos un ejemplo específico

Podemos decir que como nuestro k en todos los algoritmos está en un rango máximo de 10, estamos analizando el top_10 relevantes. (El valor por defecto es k=100).

En el caso del `knn_baseline` obtiene por ejemplo:

    - Precisión: 0.917285259809119
    - Recall: 0.190821119640963
    - F1: 0.315921654370031
    
Es decir, nuestro recomendador hace un 91,7% de recomendaciones relevantes y un 19% de las recomendaciones relevantes están en primeros resultados. Esto quiere decir que tenemos un buen balance, no todas las recomendaciones tienen que estar de primeras y aunque no da 100% relevantes, hay una gran cantidad de ellas y en su mayoría en los primeros puestos. 

En el caso del `knn_basic` obtiene por ejemplo:

    - Precisión: 0.946977730646871
    - Recall: 0.0354763013581841
    - F1: 0.0683905124463774
    
El recomendador hace un 94.6% de recomendaciones relevantes y solo un 3.5% están en los primeros resultados. No hay muy buen balance. De que me sirve tener buenas recomendaciones si no las muestro de primero aunque sea unas pocas. Conocemos que las personas prefieren ver primero recomendaciones que sí les gusten, no ir a ver la siguientes. Por eso esto no tiene un buen equilibrio. 